# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM 
from nitools.models import LeNet5, ResNet18

import numpy as np
import torch
import time as t

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [2]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

### pAE-RVFL

In [14]:
device = torch.device("cpu")

pae_rvfl = PAE_RVFL.PAE_RVFL(
    784, 
    200, 
    mnist_class, 
    subnets=5,
    device=device, 
    r=(1,1), sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5(pae_rvfl, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:10000]
y = torch.from_numpy(mnist['train_y']).float()[:10000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

train: 9.880131244659424
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.962 P=0.961 R=0.962 F1=0.961
              precision    recall  f1-score   support

         0.0      0.983     0.961     0.972      1002
         1.0      0.989     0.971     0.980      1156
         2.0      0.960     0.957     0.958      1036
         3.0      0.959     0.953     0.956      1017
         4.0      0.953     0.979     0.966       956
         5.0      0.960     0.975     0.967       878
         6.0      0.968     0.974     0.971       952
         7.0      0.950     0.948     0.949      1031
         8.0      0.955     0.958     0.956       971
         9.0      0.935     0.942     0.938      1001

    accuracy                          0.962     10000
   macro avg      0.961     0.962     0.961     10000
weighted avg      0.962     0.962     0.962     10000


Confusion matrix:
 [[ 963    1    1    1    1    2    3    4    4    0]
 [   0 1123    3    3    0   

### ELM

In [5]:
# Get ELM results
mnist_elm = ELM.ELM(
    input_size=mnist_in,
    h_size=2500,
    output_size=mnist_class)

lenet = LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

train: 90.78947496414185
Evaluation for: lenet PAE-RVFL - MNIST
Classifier 'lenet PAE-RVFL - MNIST' has Acc=0.972 P=0.972 R=0.972 F1=0.972
              precision    recall  f1-score   support

         0.0      0.991     0.971     0.981      1000
         1.0      0.989     0.977     0.983      1150
         2.0      0.979     0.971     0.975      1040
         3.0      0.974     0.969     0.971      1016
         4.0      0.964     0.983     0.974       963
         5.0      0.973     0.974     0.974       891
         6.0      0.975     0.982     0.979       951
         7.0      0.942     0.969     0.955       999
         8.0      0.976     0.981     0.979       969
         9.0      0.954     0.943     0.949      1021

    accuracy                          0.972     10000
   macro avg      0.972     0.972     0.972     10000
weighted avg      0.972     0.972     0.972     10000


Confusion matrix:
 [[ 971    1    0    1    0    2    2    1    2    0]
 [   0 1123    4    0    1   

In [ ]:
# Show results

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [ ]:
# Import dataset
cifar10 = utils.load_cifar10()

cifar_in = 32*32*3
cifar_class = 10

### SPAE-RVFL

In [ ]:
device = torch.device("cpu")

pae_rvfl = SPAE_RVFL.SPAE_RVFL(
    cifar_in, 
    cifar_class, 
    [100], 
    subnets=[5], 
    device=device, 
    r=[(1,1), (1,1)], sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5_3D(pae_rvfl)

X = torch.from_numpy(cifar10['train_X']).float()[:5000]
y = torch.from_numpy(cifar10['train_y']).float()[:5000]

result = lenet.train(X, y)
tX = torch.from_numpy(cifar10['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, cifar10['test_y'])

In [ ]:
# Show results